<a href="https://colab.research.google.com/github/chrnlee/chrnlee.github.io/blob/master/%08Drive_files_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
!pip install PyPDF2 fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=a1d058c5d078a0ca2819bb23a41bb8b9abc22dba8d936ad316ae576f406a2dff
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [32]:
# 최종
from __future__ import print_function
import os
import pickle
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from IPython.display import display, HTML, Javascript
import google.auth.transport.requests
import io
from googleapiclient.http import MediaIoBaseDownload
from fpdf import FPDF
from google.colab import files


# 인증 설정
SERVICE_ACCOUNT_FILE = '/content/primeval-array-424004-j5-4ce14e761872.json'
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=creds)
SCOPES = ['https://www.googleapis.com/auth/drive']

def authenticate():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            auth.authenticate_user()
            creds, _ = google.auth.default(scopes=SCOPES)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def list_files_within_date_range(service, start_date, end_date):
    query = f"modifiedTime >= '{start_date}' and modifiedTime <= '{end_date}'"
    results = service.files().list(
        q=query,
        fields="nextPageToken, files(id, name, mimeType, modifiedTime)").execute()
    items = results.get('files', [])
    return items

def generate_html(files):
    html = """
    <html>
    <body>
        <div id="form-container">
            <h1>Select Date Range</h1>
            <form id="dateForm">
                Start Date: <input type="date" id="startDate" name="startDate">
                End Date: <input type="date" id="endDate" name="endDate">
                <button type="button" onclick="submitDateForm()">Get Files</button>
            </form>
        </div>
        <div id="files-container">
            <h1>Files</h1>
            <form id="fileForm">
    """
    for file in files:
        html += f"""
        <div>
            <input type="checkbox" name="files" value="{file['id']}">
            <label>{file['name']} ({file['modifiedTime']})</label>
        </div>
        """
    html += """
            <button type="button" onclick="submitFileForm()">Submit</button>
            </form>
        </div>
        <div id="download-links"></div>
        <script>
            function submitDateForm() {
                var startDate = document.getElementById('startDate').value;
                var endDate = document.getElementById('endDate').value;
                google.colab.kernel.invokeFunction('notebook.getFilesByDate', [startDate, endDate], {});
            }
            function submitFileForm() {
                var form = document.getElementById('fileForm');
                var formData = new FormData(form);
                var selectedFiles = [];
                for (var pair of formData.entries()) {
                    selectedFiles.push(pair[1]);
                }
                google.colab.kernel.invokeFunction('notebook.getSelectedFiles', [selectedFiles], {});
            }
        </script>
    </body>
    </html>
    """
    return html

def update_html(files):
    html = """
    <div>
        <h1>Files</h1>
        <form id="fileForm">
    """
    for file in files:
        html += f"""
        <div>
            <input type="checkbox" name="files" value="{file['id']}">
            <label>{file['name']} ({file['modifiedTime']})</label>
        </div>
        """
    html += """
            <button type="button" onclick="submitFileForm()">Submit</button>
        </form>
    </div>
    """
    display(Javascript(f"""
        document.getElementById('files-container').innerHTML = `{html}`;
    """))

def download_files(service, file_ids):
    file_contents = []
    for file_id in file_ids:
        file = service.files().get(fileId=file_id, fields="mimeType, name").execute()
        mime_type = file.get('mimeType')
        file_name = file.get('name')
        request = None
        if mime_type == 'application/vnd.google-apps.document':
            request = service.files().export_media(fileId=file_id, mimeType='application/pdf')
        elif mime_type.startswith('text/'):
            request = service.files().get_media(fileId=file_id)

        if request:
            file_buffer = io.BytesIO()
            downloader = MediaIoBaseDownload(file_buffer, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
            file_buffer.seek(0)
            file_contents.append((file_name, file_buffer.read()))
    return file_contents

def convert_to_pdf(file_name, content):
    if isinstance(content, bytes):
        return content
    else:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, content.decode('utf-8'))
        pdf_buffer = io.BytesIO()
        pdf.output(pdf_buffer)
        pdf_buffer.seek(0)
        return pdf_buffer.read()

def get_selected_files(selected_files):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    file_contents = download_files(service, selected_files)

    for file_name, content in file_contents:
        pdf_content = convert_to_pdf(file_name, content)
        pdf_file_path = f"/content/{file_name}.pdf"
        with open(pdf_file_path, 'wb') as f:
            f.write(pdf_content)
        files.download(pdf_file_path)

def get_files_by_date(start_date, end_date):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    files = list_files_within_date_range(service, start_date, end_date)
    update_html(files)

from google.colab import output
output.register_callback('notebook.getSelectedFiles', get_selected_files)
output.register_callback('notebook.getFilesByDate', get_files_by_date)

def main():
    html_content = generate_html([])
    display(HTML(html_content))

if __name__ == '__main__':
    main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>